<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Regressão-Linear-Múltipla" data-toc-modified-id="Regressão-Linear-Múltipla-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Regressão Linear Múltipla</a></span><ul class="toc-item"><li><span><a href="#Simulando-Dados" data-toc-modified-id="Simulando-Dados-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Simulando Dados</a></span><ul class="toc-item"><li><span><a href="#Duas-variáveis-continuas" data-toc-modified-id="Duas-variáveis-continuas-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Duas variáveis continuas</a></span></li><li><span><a href="#Simulando-Variáveis-Categóricas" data-toc-modified-id="Simulando-Variáveis-Categóricas-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Simulando Variáveis Categóricas</a></span><ul class="toc-item"><li><span><a href="#Convertendo-variáveis-categóricas-em-variáveis-dummy" data-toc-modified-id="Convertendo-variáveis-categóricas-em-variáveis-dummy-1.1.2.1"><span class="toc-item-num">1.1.2.1&nbsp;&nbsp;</span>Convertendo variáveis categóricas em variáveis dummy</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Voltamos-20h20" data-toc-modified-id="Voltamos-20h20-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Voltamos 20h20</a></span><ul class="toc-item"><li><span><a href="#Dados-Reais" data-toc-modified-id="Dados-Reais-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Dados Reais</a></span></li><li><span><a href="#Exploração-de-Dados" data-toc-modified-id="Exploração-de-Dados-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Exploração de Dados</a></span></li><li><span><a href="#Criando-nossa-regressão" data-toc-modified-id="Criando-nossa-regressão-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Criando nossa regressão</a></span></li></ul></li><li><span><a href="#VOLTAMOS-21h15" data-toc-modified-id="VOLTAMOS-21h15-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>VOLTAMOS 21h15</a></span></li></ul></div>

In [ ]:
import random
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Regressão Linear Múltipla

## Simulando Dados

Começaremos vendo como a entrada de múltiplas variáveis em uma regressão linear funciona a partir de conjuntos de dados sintéticos.

### Duas variáveis continuas

In [ ]:
def simular_dado_mv(parametros_x1, parametros_x2, desvpad_E, samples):
    x1 = np.random.normal(loc=parametros_x1[0], scale=parametros_x1[1], size=samples)

    x2 = np.random.normal(loc=parametros_x2[0], scale=parametros_x2[1], size=samples)

    E = np.random.normal(loc=0, scale=desvpad_E, size=samples)

    y = parametros_x1[2] * x1 + parametros_x2[2] * x2 + E

    return pd.DataFrame({"x1": x1, "x2": x2, "y": y})

Podemos criar um DataFrame com nossos dados simulados para utilizarmos nossas ferramentas de EDA buscando visualizar a relação entre X1, X2 e Y

In [ ]:
teste_mv = simular_dado_mv((0, 1, 10), (0, 1, -5), 1, 100)
sns.pairplot(teste_mv)

Agora, vamos utilizar nossa regressão para estimar os coeficientes associados à X1 e X2

In [ ]:
X = sm.add_constant(teste_mv[["x1", "x2"]])
Y = teste_mv["y"]
modelo = sm.OLS(Y, X)
lm_fit = modelo.fit()
lm_fit.summary()

Podemos utilizar a biblioteca `sklearn` para conseguir a interface preditiva desta biblioteca:

In [ ]:
modelo = LinearRegression()

X = teste_mv[["x1", "x2"]]

y = teste_mv["y"]

modelo.fit(X, y)

print(modelo.coef_)
print(modelo.intercept_)

Vamos criar uma nova coluna em nosso DataFrame para conter nossas previsões:

In [ ]:
teste_mv["pred"] = modelo.predict(teste_mv[["x1", "x2"]])

In [ ]:
teste_mv

Agora, vamos comparar as relações de nossa variável resposta e variável prevista contra X1 e X2:

In [ ]:
sns.pairplot(teste_mv)

### Simulando Variáveis Categóricas

Muitas variáveis que encontramos são **categóricas**, ou seja, não são numéricas. Vamos simular uma variável categórica com impacto linear sobre nossa variável Y:

In [ ]:
def simular_dado_mv_cat(parametros_x1, categorias_dict, desvpad_E, samples):
    x1 = np.random.normal(loc=parametros_x1[0], scale=parametros_x1[1], size=samples)

    cat = random.choices(list(categorias_dict.keys()), k=samples)
    eff_cat = list(map(lambda x: categorias_dict[x], cat))

    E = np.random.normal(loc=0, scale=desvpad_E, size=samples)

    y = parametros_x1[2] * x1 + eff_cat + E
    return pd.DataFrame({"x1": x1, "categoria": cat, "y": y})

In [ ]:
simular_dado_mv_cat((0, 1, 5), {"A": 1, "B": 5, "C": 10}, 1, 100)

In [ ]:
teste_mv_cat = simular_dado_mv_cat((0, 1, 5), {"A": 1, "B": 5, "C": 10}, 1, 100)

Podemos utilizar um BoxPlot para visualizar a relação entre nossa variável categórica e nossa variável resposta:

In [ ]:
sns.boxplot(data=teste_mv_cat, x="categoria", y="y")

ou então alterar a cor dos pontos em um scatterplot, visualizando múltiplas relações ao mesmo tempo:

In [ ]:
sns.scatterplot(data=teste_mv_cat, x="x1", y="y", hue="categoria")

#### Convertendo variáveis categóricas em variáveis dummy

Para utilizarmos variáveis categóricas em um modelo, precisamos transforma-las em variáveis **numéricas**. A forma mais simples de fazê-lo é através de **variáveis dummies**: vamos criar **uma variável binaria nova para cada nível de nossa variável categórica**. Em cada variável, esta terá valor 1 quando a observação for daquela categoria e 0 caso não.

Vamos utilizar um método da Sklearn para realizar essa transformação: o `OneHotEncoder`.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder(sparse=False, drop="first")
ohe.fit(teste_mv_cat[["categoria"]])

Agora, vamos utilizar o método `.transform()` para obter nossas variáveis dummies:

In [ ]:
ohe.transform(teste_mv_cat[["categoria"]])

Para saber qual coluna é qual nível da categoria, podemos utilizar o método `.get_feature_names_out()`:

In [ ]:
ohe.get_feature_names_out()

Vamos combinar esses dois métodos para criar um DataFrame com nossas variáveis dummies:

In [ ]:
tb_dummies = pd.DataFrame(
    ohe.transform(teste_mv_cat[["categoria"]]),
    columns = ohe.get_feature_names_out()
)
tb_dummies.head()

In [ ]:
tb_teste = pd.concat([teste_mv_cat, tb_dummies], axis=1)

Vamos utilizar nossa nova tabela de dados para criar nossa regressão:

In [ ]:
X = tb_teste[["x1", "categoria_B", "categoria_C"]]
y = tb_teste["y"]

lm_fit = LinearRegression()
lm_fit.fit(X, y)

E o método `.predict()` para criar uma variável com nossas previsões:

In [ ]:
tb_teste['pred'] = lm_fit.predict(X)

In [ ]:
sns.scatterplot(data = tb_teste, x = "x1", y = "y", hue = "categoria")
sns.lineplot(data = tb_teste, x = "x1", y = "pred", hue = "categoria")

Como podemos interpretar os coeficientes das categorias B e C? O que aconteceu com a categoria A?

In [ ]:
print(list(zip(lm_fit.feature_names_in_, lm_fit.coef_)))

In [ ]:
print(lm_fit.intercept_)

## Transformações não-lineares

A regressão linear é um método estatístico usado para encontrar uma relação entre duas variáveis, onde uma variável é a "variável dependente" e a outra é a "variável independente". Na regressão linear com transformação de variáveis, estamos procurando por um relacionamento entre essas duas variáveis que não seja necessariamente linear. 

Por exemplo, se temos dados de altura e peso, podemos usar a regressão linear para ver como o peso está relacionado à altura - mas, às vezes, essa relação nem sempre parece seguir uma linha reta. É aí que entram as transformações de variáveis: podemos aplicar uma função matemática (como o logaritmo, por exemplo) a uma ou ambas as variáveis para tentar encontrar uma relação mais clara entre elas.

As transformações nos permitem ajustar os dados a um modelo mais adequadamente e, assim, fazer previsões mais precisas. No geral, a regressão linear com transformação de variáveis é usada para entender melhor a relação entre duas variáveis e fazer previsões mais precisas com base nessa relação. Espero que isso ajude!

In [ ]:
def simular_dado_nl(parametros_x1, desvpad_E, samples):
    x1 = np.random.normal(loc=parametros_x1[0], scale=parametros_x1[1], size=samples)

    E = np.random.normal(loc=0, scale=desvpad_E, size=samples)

    y = np.exp(parametros_x1[2] * x1  + E)

    return pd.DataFrame({"x1": x1, "y": y})

In [ ]:
tb_nl = simular_dado_nl((1, 0.3, 2.5), 0.35, 1000)

Vamos visualizar a relação entre X e y:

In [ ]:
sns.scatterplot(data = tb_nl, x = "x1", y = "y")

Claramente existe uma relação entre x1 e y, mas ela pode ser representada por uma linha reta?

In [ ]:
X = tb_nl[["x1"]]
y = tb_nl["y"]
lm_fit = LinearRegression()
lm_fit.fit(X, y)

tb_nl["pred_lin"] = lm_fit.predict(X)


In [ ]:
sns.scatterplot(data = tb_nl, x = "x1", y = "y")
sns.lineplot(data = tb_nl, x = "x1", y = "pred_lin", color = "red")

O gráfico acima apresenta duas *"evidências"* típicas de uma transformação específica:

* O efeito de x1 sobre y parece *crescer* conforme x1 aumenta - a curva apresenta um crescimento *exponencial*;
* A dispersão ao redor dessa curva imaginária *aumenta* conforme x1 (e y) aumenta.

Essas duas evidências (não-linearidade e heterosquedasticidade) são evidêcias de uma transformação logarítmica - x1 tem uma relação linear com o logaritmo de y! Vamos aplicar essa transformação em nosso modelo:

In [ ]:
tb_nl["log_y"] = np.log(tb_nl["y"])
sns.scatterplot(data = tb_nl, x = "x1", y = "log_y")

Vamos construir nossa regressão através dessa nova variável y:

In [ ]:
X = tb_nl[["x1"]]
y = tb_nl["log_y"]
lm_fit = LinearRegression()
lm_fit.fit(X, y)

tb_nl["pred_log"] = lm_fit.predict(X)


In [ ]:
sns.scatterplot(data = tb_nl, x = "x1", y = "log_y")
sns.lineplot(data = tb_nl, x = "x1", y = "pred_log")

Agora nossa regressão está bem ajustada! Mas nossas previsões estão em logaritmo, como podemos voltar para a escala original?

In [ ]:
tb_nl["pred_exp"] = np.exp(tb_nl["pred_log"])
sns.scatterplot(data = tb_nl, x = "x1", y = "y")
sns.lineplot(data = tb_nl, x = "x1", y = "pred_exp", color = "red")


Por último, vamos olhar duas outras transformações importantes:
1. A relação onde x1 é inversamente proporcional ao logaritmo de y,
1. a relação onde o logaritmo de x1 é diretamente proporcional à y.

In [ ]:
tb_nl_inv = simular_dado_nl((1, 0.3, -2.5), 0.35, 1000)
sns.scatterplot(data = tb_nl_inv, x = "x1", y = "y")

In [ ]:
def simular_dado_nl_2(parametros_x1, desvpad_E, samples):
    x1 = np.random.normal(loc=parametros_x1[0], scale=parametros_x1[1], size=samples)

    E = np.random.normal(loc=0, scale=desvpad_E, size=samples)

    y = parametros_x1[2] * np.exp(x1)  + E

    return pd.DataFrame({"x1": x1, "y": y})

In [ ]:
tb_nl_inv = simular_dado_nl_2((1, 0.3, 2.5), 0.35, 1000)
sns.scatterplot(data = tb_nl_inv, x = "x1", y = "y")

Além de transformações logarítmicas, temos outras transformações menos comuns como a quadrática, ou a exponencial. O importante é lembrarmos que **qualquer** operação válida em uma variável pode ser utilizada para transforma-la - e que a escolha da transformação correta é uma parte importante do processo de modelagem. Muitas vezes, a transformação correta é encontrada através de tentativa e erro e de análise exploratória.

Devemos sempre nos lembrar que as transformações sobre a variável resposta alteram a relação desta com todas as nossas variáveis X - e que alteram o **domínio**, ou seja, o range de valores que nossa variável y poderá assumir. No caso da transformação logarítimica de y temos que y > 0, por exemplo.

# Trabalhando exemplos reais
## Prevendo Consumo de Combustível

In [ ]:
tb_autompg = pd.read_csv("data/tb_autompg.csv")

In [ ]:
sns.pairplot(tb_autompg[["mpg", "cylinders", "displacement", "horsepower"]])

## Prevendo Aluguéis de Bicicleta

In [ ]:
tb_bike = pd.read_csv("data/tb_bikesharing.csv")

In [ ]:
tb_bike.describe()